In [1]:
import numpy as np
import pandas as pd
import pandas_datareader as pdr
import plotly.graph_objects as go
from sklearn.linear_model import LinearRegression

In [3]:
# ERi = Rf + beta(ERm - Rf)
# Rf => is 10 Year Treasury Return (low value)
# ERm => SP500 expected return (10.5)

In [5]:
sp500_stock = pdr.get_data_yahoo("^GSPC", "1926", "2014")
sp500_stock

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
1927-12-30,17.660000,17.660000,17.660000,17.660000,0,17.660000
1928-01-03,17.760000,17.760000,17.760000,17.760000,0,17.760000
1928-01-04,17.719999,17.719999,17.719999,17.719999,0,17.719999
1928-01-05,17.549999,17.549999,17.549999,17.549999,0,17.549999
1928-01-06,17.660000,17.660000,17.660000,17.660000,0,17.660000
...,...,...,...,...,...,...
2013-12-24,1833.319946,1828.020020,1828.020020,1833.319946,1307630000,1833.319946
2013-12-26,1842.839966,1834.959961,1834.959961,1842.020020,1982270000,1842.020020
2013-12-27,1844.890015,1839.810059,1842.969971,1841.400024,2052920000,1841.400024


In [6]:
sp500_log_returns = np.log(sp500_stock / sp500_stock.shift())
sp500_log_returns

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
1927-12-30,NaN,NaN,NaN,NaN,NaN,NaN
1928-01-03,0.005647,0.005647,0.005647,0.005647,NaN,0.005647
1928-01-04,-0.002255,-0.002255,-0.002255,-0.002255,NaN,-0.002255
1928-01-05,-0.009640,-0.009640,-0.009640,-0.009640,NaN,-0.009640
1928-01-06,0.006248,0.006248,0.006248,0.006248,NaN,0.006248
...,...,...,...,...,...,...
2013-12-24,0.001949,0.002794,0.002794,0.002912,-0.779643,0.002912
2013-12-26,0.005179,0.003789,0.003789,0.004734,0.416026,0.004734
2013-12-27,0.001112,0.002640,0.004356,-0.000337,0.035021,-0.000337


In [9]:
cumlative_return = np.sum(sp500_log_returns) - 1
cumlative_return

High         3.651336
Low          3.647528
Open              NaN
Close        3.650752
Volume            inf
Adj Close    3.650752
dtype: float64

In [8]:
tickers = ["AAPL", "TWTR", "IBM", "MSFT", "^GSPC"]
stocks_price = pdr.get_data_yahoo(tickers, "2015-12-01", "2021-01-01", interval="m")["Adj Close"]
stocks_price.head()

Symbols,AAPL,TWTR,IBM,MSFT,^GSPC
Date,,,,,
2015-12-01,24.266083,23.139999,106.805664,50.298275,2043.939941
2016-01-01,22.440247,16.799999,96.848419,49.944706,1940.239990
2016-02-01,22.290400,18.120001,101.691223,46.127903,1932.229980
2016-03-01,25.262314,16.549999,118.739609,50.431122,2059.739990
2016-04-01,21.727581,14.620000,114.419662,45.536850,2065.300049


In [10]:
log_returns = np.log(stocks_price / stocks_price.shift())
log_returns

Symbols,AAPL,TWTR,IBM,MSFT,^GSPC
Date,,,,,
2015-12-01,NaN,NaN,NaN,NaN,NaN
2016-01-01,-0.078224,-0.320184,-0.097864,-0.007054,-0.052068
2016-02-01,-0.006700,0.075638,0.048794,-0.079498,-0.004137
2016-03-01,0.125158,-0.090630,0.154992,0.089190,0.063905
2016-04-01,-0.150731,-0.123996,-0.037060,-0.102087,0.002696
...,...,...,...,...,...
2020-09-01,-0.106370,0.092214,-0.000381,-0.067361,-0.040018
2020-10-01,-0.061888,-0.073175,-0.085854,-0.038086,-0.028056
2020-11-01,0.089481,0.117353,0.100944,0.055711,0.102146


In [11]:
cov = log_returns.cov()
var = log_returns["^GSPC"].var()
beta = cov["^GSPC"] / var
beta

Symbols
AAPL     1.254557
TWTR     0.933242
IBM      1.249102
MSFT     0.797269
^GSPC    1.000000
Name: ^GSPC, dtype: float64

In [12]:
risk_free_return = 0.0230
market_return = 0.105

In [13]:
expected_return = risk_free_return + beta *(market_return - risk_free_return)
expected_return

Symbols
AAPL     0.125874
TWTR     0.099526
IBM      0.125426
MSFT     0.088376
^GSPC    0.105000
Name: ^GSPC, dtype: float64